In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

# CSV 파일 읽기
data = pd.read_csv('../data/pm25_pm10_merged.csv')

# 날짜를 datetime 형식으로 변환 및 연도 추출
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year

# 미세먼지 등급 정의 (PM2.5 기준)
def classify_pm25(pm25):
    if pm25 <= 15:
        return '좋음'
    elif pm25 <= 35:
        return '보통'
    elif pm25 <= 75:
        return '나쁨'
    else:
        return '매우 나쁨'

data['PM2.5 Grade'] = data['PM2.5 (µg/m³)'].apply(classify_pm25)

# 중국(Beijing)의 PM2.5 농도를 주요 feature로 설정
data['Beijing_PM2.5'] = data[data['City'] == 'Beijing']['PM2.5 (µg/m³)']

# 주변국 데이터 필터링 (서울, 도쿄, 방콕 등)
neighbor_cities = ['Seoul', 'Tokyo', 'Bangkok']
neighbor_data = data[data['City'].isin(neighbor_cities)]

# Feature와 Target 설정
features = neighbor_data[['Beijing_PM2.5', 'Longitude', 'Latitude', 'Year']]
target = neighbor_data['PM2.5 Grade']

# 결측값 제거
features.fillna(0, inplace=True)

# 레이블 인코딩 (등급을 숫자로 변환)
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)

# 데이터 분할 (훈련/테스트 세트)
X_train, X_test, y_train, y_test = train_test_split(features, target_encoded, test_size=0.2, random_state=42)

# XGBoost 모델 생성 및 학습
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

# 예측 및 평가
y_pred = xgb_model.predict(X_test)

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# 분류 보고서 출력
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# 예측된 등급을 원래 라벨로 변환
predicted_labels = label_encoder.inverse_transform(y_pred)
true_labels = label_encoder.inverse_transform(y_test)

# 예측 결과 확인
result_df = pd.DataFrame({'실제 등급': true_labels, '예측 등급': predicted_labels})
print("\n예측 결과 (일부):\n", result_df.head())


C:\Users\Playdata\AppData\Local\Temp\ipykernel_908\2101867821.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(0, inplace=True)
c:\Users\Playdata\anaconda3\envs\pystudy_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:18:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix:
 [[ 25  43 295  27]
 [ 26  96  67  16]
 [ 18   7 670  94]
 [  2   3 270 570]]

Classification Report:
               precision    recall  f1-score   support

          나쁨       0.35      0.06      0.11       390
       매우 나쁨       0.64      0.47      0.54       205
          보통       0.51      0.85      0.64       789
          좋음       0.81      0.67      0.73       845

    accuracy                           0.61      2229
   macro avg       0.58      0.51      0.51      2229
weighted avg       0.61      0.61      0.57      2229


예측 결과 (일부):
   실제 등급 예측 등급
0    좋음    좋음
1    좋음    좋음
2    좋음    보통
3    보통    보통
4    좋음    좋음
